# core

> This is a proofs validator to help students and hobbyists do mathematical thinking and problem solving.
>
> It's for when you buy a math book from the local used book store, so you have a piece of software to use to explore the mathematical concepts alongside you that's fun and easy to use for anybody with a programming background. It's to help amateurs satisfy their mathematical curiosity.
>
> It's supposed to validate what you're doing in a "black box" kind of manner, and try to offer you guardrails enough that you can spot your mistakes and feel *reasonably* more confident you know what you're doing.
>
> Not perfectly confident, reasonably.


In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from typing import Callable, List, Tuple, Union
from sympy import Equality, Unequality
from sympy.core.relational import Relational
from sympy.core.basic import Basic
from random import randint, seed
import sympy as sp

from IPython.display import display, Latex

import typing
import inspect

In [ ]:
#| export
# Basic types
Var = sp.Symbol
Const = sp.Number
Func = Callable[[Var], sp.Expr]
Goal = Union[Equality, Unequality, Relational]
Expression = Union[sp.Expr, Basic]

# TODO - build in a repr override to use latex

# Helper functions to create a more user-friendly interface
def variable(name: str) -> Var:
    return sp.Symbol(name)

def constant(value: Union[int, float]) -> Const:
    return sp.Number(value)

def equation(expr: str) -> sp.Expr:
    return sp.sympify(expr, evaluate=False)

def equals(lhs: sp.Expr, rhs: sp.Expr) -> Equality:
    return sp.Eq(lhs, rhs, evaluate=False)

def not_equals(lhs: Expression, rhs: Expression) -> Unequality:
    return sp.Ne(lhs, rhs, evaluate=False)

In [ ]:
#| export
def make_examples(domain: str, N: int, equation: str) -> List[Tuple[sp.Expr, sp.Expr]]:
    """For a given domain and equation, select N examples and generate a list of N input-output pairs.
    Currently, the domain can be either 'real' or 'integer', and one variable is assumed."""
    examples = []
    if domain == 'real':
        # select N random real numbers
        for _ in range(N):
            x = sp.Symbol('x')
            x_val = randint(-100, 100)
            y_val = sp.sympify(equation).subs(x, x_val)
            examples.append((x_val, y_val))
    elif domain == 'integer':
        # select N random integers
        for _ in range(N):
            x = sp.Symbol('x', integer=True)
            x_val = randint(-100, 100)
            y_val = sp.sympify(equation).subs(x, x_val)
            examples.append((x_val, y_val))
    return examples

In [ ]:
make_examples('real', 10, 'x**2')

[(-13, 169),
 (-29, 841),
 (-61, 3721),
 (-45, 2025),
 (95, 9025),
 (-14, 196),
 (-74, 5476),
 (-77, 5929),
 (-3, 9),
 (-76, 5776)]

In [ ]:
#| export
def prove(goal: Goal, proof_func: Callable[..., Goal], *args) -> bool:
    try:
        derived_goal = proof_func(*args)
        if goal == derived_goal:
            display(Latex(f"$$\\text \\quad {sp.latex(goal)} \\quad Q.E.D.$$"))
            return True
    except Exception as e:
        print(f"Proof error: {str(e)}")
        print("Check your assumptions and proof function for errors.")
        return False

In [ ]:
#| export
def contradiction_proof(proof):
    def wrapper(*args, **kwargs):
        hints = typing.get_type_hints(proof)
        if hints != {}:
            if hints.get('return') != Unequality:
                print(hints.get('return'))
                raise TypeError("Proof function must return Unequality")
        else:
            try:
                # add all of the arguments to the local namespace with their existing names that are passed in.
                args_list = lambda args: [_arg for _arg in args]
                _printed = ['_i', '_arg', 'line', 'var_name','args_list', '_printed', 'var_value', 'proof', 'args', 'kwargs', 'hints']
                for _i, _arg in enumerate(args_list(args)):
                    exec(f"{_arg} = args[{_i}]", globals(), locals())
                    _printed.append(str(_arg))
                for _i, line in enumerate(inspect.getsourcelines(proof)[0]):
                    #print each varaible only once, and add opt outs
                    line = line.strip()
                    if line.startswith('#'):
                        # in future concatenate all the strings into one doc and format nicely.
                        # first attempt didn't work out, had weird formatting issues.
                        display(Latex(f"{line[1:].strip()}"))
                        continue
                    if line.startswith('def'):
                        continue
                    if line.startswith('@'):
                        continue
                    if line.startswith('return'):
                        line = line[7:]
                    if line == '':
                        continue
                    exec(line, globals(), locals())
                    # print(f"\nProof state after line {i+1}: {line}")
                    for var_name, var_value in locals().items():
                        if var_name in _printed:
                            continue
                        else:
                            _printed.append(var_name)
                        # if isinstance(var_value, (Var, Const, Func)):
                        display(Latex(f"$${sp.latex(var_value)}$$"))
            except Exception as e:
                print(f"Error in proof function: {str(e)}")
                raise
            # there might be something weird around result that would prevent it from being printed if it's in the proof. check that later
            result = proof(*args, **kwargs)
            if not isinstance(result, Unequality):
                raise TypeError("Proof function must return Unequality")
            return result
    return wrapper

Here's how to use a contradiction goal.

In [ ]:
# Start by defining your domain
arbitrary_x = variable("x")
expression = arbitrary_x + 1

# Then define your goal
contradiction_goal = not_equals(expression, arbitrary_x)

@contradiction_proof
def proof_of_x_plus_one(x):
    # Given x, Assume x + 1 = x is true for arbitrary_x
    assumed_eq = equals(x + 1, x)

    # Calculate x + 1
    next = x + 1

    # Observing x + 1 $\neq$ x, we have reached a contradiction
    return not_equals(next, assumed_eq.rhs)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
prove(contradiction_goal, proof_of_x_plus_one, arbitrary_x)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()